In [2]:
pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

import json
from sentence_transformers import SentenceTransformer, util
import concurrent.futures
from tqdm import tqdm

C:\Users\David Yu\SkillgapAnalyis2\SkillGapAnalysis\Backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
if __name__ == "__main__":
    model = SentenceTransformer("./models/fine_tuned_sentence_transformer")

    # Load the skills JSON
    with open("../data/new_skills.json", "r") as f:
        skills_data = json.load(f)

    skill_names = [skill['name'] for skill in skills_data['skills']]
    skill_embeddings = model.encode(skill_names, convert_to_tensor=True)

    # Define a picklable function for parallel similarity matching
    def find_synonyms(skill_idx):
        similarities = util.pytorch_cos_sim(skill_embeddings[skill_idx], skill_embeddings)[0]
        synonyms = [skill_names[i] for i in range(len(skill_names)) 
                    if similarities[i].item() > 0.60 and i != skill_idx]
        return synonyms

    # Parallelize with tqdm progress bar
    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = list(tqdm(executor.map(find_synonyms, range(len(skill_names))),
                            total=len(skill_names), desc="Expanding Skills"))

    # Update the skill set with synonyms
    for idx, skill in enumerate(skills_data['skills']):
        skill['synonyms'] = results[idx]

    # Save the expanded skills list
    with open("expanded_skills_parallel.json", "w") as f:
        json.dump(skills_data, f, indent=2)

    print("✅ Skill expansion completed with parallel processing!")

NameError: name 'SentenceTransformer' is not defined